###Importing libraries

In [1]:
import numpy as np
import pandas as pd

###Importing dataset (from google drive)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/1_LiveProjects/Project2_Sentiment_Analysis
!ls

/content/drive/MyDrive/1_LiveProjects/Project2_Sentiment_Analysis
a1_RestaurantReviews_HistoricDump.tsv  c1_BoW_Sentiment_Model.pkl
a2_RestaurantReviews_FreshDump.tsv     c2_Classifier_Sentiment_Model
b1_Sentiment_Analysis_Model.ipynb      c3_Predicted_Sentiments_Fresh_Dump.tsv
b2_Sentiment_Predictor.ipynb	       d_Business_Deck_Sentiment_Analysis.pdf


In [4]:
dataset = pd.read_csv('a2_RestaurantReviews_FreshDump.tsv', delimiter = '\t', quoting = 3)
dataset.head()

,Review,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Spend your money elsewhere.,NaN,NaN,NaN
1,Their regular toasted bread was equally satisf...,NaN,NaN,NaN
2,The Buffet at Bellagio was far from what I ant...,NaN,NaN,NaN
3,And the drinks are WEAK,people!,NaN,NaN
4,-My order was not correct.,NaN,NaN,NaN


###Data cleaning

In [5]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
corpus=[]

for i in range(0, 100):
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

###Data transformation

In [7]:
# Loading BoW dictionary
from sklearn.feature_extraction.text import CountVectorizer
import pickle
cvFile='c1_BoW_Sentiment_Model.pkl'
# cv = CountVectorizer(decode_error="replace", vocabulary=pickle.load(open('./drive/MyDrive/Colab Notebooks/2 Sentiment Analysis (Basic)/3.1 BoW_Sentiment Model.pkl', "rb")))
cv = pickle.load(open(cvFile, "rb"))


In [12]:
X_fresh = cv.transform(corpus).toarray()
X_fresh.shape

(100, 1420)

###Predictions (via sentiment classifier)

In [13]:
import joblib
classifier = joblib.load('c2_Classifier_Sentiment_Model')

In [14]:
y_pred = classifier.predict(X_fresh)
print(y_pred)

[0 1 1 0 0 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0
 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0]


In [15]:
dataset['predicted_label'] = y_pred.tolist()
dataset.head()

,Review,Unnamed: 1,Unnamed: 2,Unnamed: 3,predicted_label
0,Spend your money elsewhere.,NaN,NaN,NaN,0
1,Their regular toasted bread was equally satisf...,NaN,NaN,NaN,1
2,The Buffet at Bellagio was far from what I ant...,NaN,NaN,NaN,1
3,And the drinks are WEAK,people!,NaN,NaN,0
4,-My order was not correct.,NaN,NaN,NaN,0


In [17]:
dataset.to_csv("c3_Predicted_Sentiments_Fresh_Dump.tsv", sep='\t', encoding='UTF-8', index=False)